# GAN 실험해보기
- Pytorch로 GAN 구조를 짜보고, MNIST digit으로 학습하여, gan 의 generator 가 제대로 동작하는지 확인해보겠습니다.

### 1. Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

from generator import Generator
from discriminator import Discriminator

import matplotlib.pyplot as plt

# device setting for gpu users
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device: ", device)

### 2. Data Preparation
MNIST digit data 를 활용하겠습니다.

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize(mean=(0.5, 0.5, 0.5), 
                                std=(0.5, 0.5, 0.5))])

mnist_dataset = datasets.MNIST(root='./data/', train=True, transform=transform, download=True)

dataloader = DataLoader(mnist_dataset, batch_size=128, shuffle=True)

### 3. Modeling
깔끔한 노트북을 위해, `discriminator.py` 와 `generator.py` 에 각 Discriminator 와 Generator를 정의해 두었습니다. 이 노트북에서는 초기화 선언만 하겠습니다.

In [ ]:
generator = Generator(latent_dims=64)
discriminator = Discriminator()
print("GENERATOR : ", generator)
print("DISCRIMINATOR : ", discriminator)

### 4. Train
#### 4-1. Loss & Optimizer
- GAN 의 구조에서부터 알 수 있듯이, 지금 네트워크는 discriminator 가 generator 로 부터 받은 생성된 사진과 실제 사진이 각각 진짜인지, 가짜인지 맞추는 loss 로 부터 역전파 되어 각 구조가 학습하게 됩니다. 따라서 discriminator 의 마지막 layer의 크기와 Bincary Cross Entropy Loss 가 구조로부터 정해지게 됩니다.
- Optimizer 의 경우, 우리는 discriminator 와 generator 가 순차적으로 학습하는 구조를 가질 수 밖에 없습니다. 따라서, 각 구조를 update 시켜주기위한 optimizer 는 따로 선언해 줍니다.

In [ ]:
# Loss & Optimizer
criterion = nn.BCELoss()
generator_optim = optim.Adam(generator.parameters(), lr=0.001)
discriminator_optim = optim.Adam(discriminator.parameters(), lr=0.001)

#### 4-2. Train Model
- discriminator 의 학습을 위해 train 단계에서, 진짜(1)와 가짜(0) 이미지의 label을 붙여줍니다.

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    for batch_images, _ in dataloader:
        images = batch_images.view(128, -1)
        
        